In [ ]:
# !pip install transformers datasets evaluate

from google.colab import drive
drive.mount('/content/drive')

!pip install transformers==4.28.0
!pip install rouge_score
!pip install datasets
!pip install evaluate
!pip install wandb
!pip install sentencepiece
!pip install transformers-interpret

from fastai.imports import *
from pathlib import Path
import os
from torch.utils.data import DataLoader
import warnings,transformers,logging,torch
from transformers import TrainingArguments,Trainer
from transformers import AutoModelForSequenceClassification,AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import datasets
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import evaluate
import numpy as np 
from sklearn import metrics
import pandas as pd 

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 128.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4c2635474d0cb66a4e19bb9572a1fdb7a5f64d272f73b6f5d6831ab1998e2098
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
# Chooses between GPU and CPU based on availability
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# text = 'There are 82 people in the community cluster who remain at the Auckland Quarantine facility that includes 59 people who have tested positive for COVID-19 and their household contacts. That number continues to decline as cases recover and reach the end of their isolation period'

text = 'Drinking a mix of baking soda and lemon as a hot tea cures COVID'

trained_model = "/content/drive/MyDrive/Information Retrieval System/Course_Project/saved_models/BERT/topic_classifier_outputs/checkpoint-4390/"

tokenizer = AutoTokenizer.from_pretrained(trained_model)
# inputs = tokenizer(text, return_tensors="pt")

model = AutoModelForSequenceClassification.from_pretrained(trained_model)

from transformers_interpret import SequenceClassificationExplainer

cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
word_attributions = cls_explainer(text)
# cls_explainer.visualize("model_trained.html")
word_attributions

[('[CLS]', 0.0),
 ('drinking', 0.6380325373333052),
 ('a', 0.4019428924614682),
 ('mix', -0.007841742150215733),
 ('of', 0.14399786889470528),
 ('baking', 0.07059746487827107),
 ('soda', 0.1941522157733687),
 ('and', -0.12568588445963289),
 ('lemon', 0.1272610699730881),
 ('as', -0.1670725313199976),
 ('a', 0.015897163620144743),
 ('hot', 0.13280931949464606),
 ('tea', 0.13476404019080027),
 ('cure', 0.21784791767281023),
 ('##s', 0.27122219189514574),
 ('co', 0.38845820885184396),
 ('##vid', -0.0018766662829276136),
 ('[SEP]', 0.0)]